![Redis](https://redis.io/wp-content/uploads/2024/04/Logotype.svg?auto=webp&quality=85,75&width=120)

# Context Fusion: Intelligent Multi-Source Integration

## Learning Objectives (40 minutes)
By the end of this notebook, you will be able to:
1. **Understand** the challenges of combining context from multiple sources
2. **Implement** intelligent context fusion strategies for conflicting information
3. **Design** priority systems for different context sources
4. **Create** coherent context from fragmented information across systems
5. **Handle** temporal conflicts and information freshness in context fusion

## Prerequisites
- Completed previous notebooks in Section 5
- Understanding of your Agent Memory Server and Redis integration
- Familiarity with context pruning and summarization techniques

---

## Introduction

**Context Fusion** is the practice of intelligently combining context from multiple sources to create a coherent, comprehensive understanding. In your Redis University system, context comes from many places:

- **Conversation History**: Current session interactions
- **Agent Memory Server**: Long-term student memories
- **Student Profile**: Academic records and preferences
- **Course Database**: Real-time course information
- **External APIs**: Career data, industry trends

### The Context Fusion Challenge

**Common Problems:**
- **Conflicting Information**: Student says "I prefer online" but profile shows "prefers in-person"
- **Temporal Misalignment**: Old preferences vs. new statements
- **Source Reliability**: Which source to trust when information conflicts
- **Information Gaps**: Incomplete data across different systems
- **Context Overload**: Too much information from too many sources

### Our Solution: Intelligent Fusion Engine

We'll implement:
1. **Source prioritization** based on recency and reliability
2. **Conflict resolution** strategies for contradictory information
3. **Temporal awareness** for handling time-sensitive context
4. **Coherence validation** to ensure fused context makes sense

## Environment Setup

In [ ]:
# Environment setup
import os
import asyncio
import json
from typing import List, Dict, Any, Optional, Tuple, Union
from dataclasses import dataclass, field
from datetime import datetime, timedelta
from enum import Enum
import uuid
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

REDIS_URL = os.getenv("REDIS_URL", "redis://localhost:6379")
AGENT_MEMORY_URL = os.getenv("AGENT_MEMORY_URL", "http://localhost:8088")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

print("🔧 Environment Setup")
print("=" * 30)
print(f"Redis URL: {REDIS_URL}")
print(f"Agent Memory URL: {AGENT_MEMORY_URL}")
print(f"OpenAI API Key: {'✅ Set' if OPENAI_API_KEY else '❌ Not set'}")

In [ ]:
# Import required modules
try:
    import redis
    from redis_context_course.models import StudentProfile, Course
    from redis_context_course.course_manager import CourseManager
    from redis_context_course.redis_config import redis_config
    
    # Redis connection
    redis_client = redis.from_url(REDIS_URL)
    if redis_config.health_check():
        print("✅ Redis connection healthy")
    else:
        print("❌ Redis connection failed")
    
    # Course manager
    course_manager = CourseManager()
    
    print("✅ Core modules imported successfully")
    
except ImportError as e:
    print(f"❌ Import failed: {e}")
    print("Please ensure you've completed the setup from previous sections.")

## Context Source Framework

Let's define a framework for managing different context sources:

In [ ]:
class ContextSourceType(Enum):
    """Types of context sources in the system."""
    CONVERSATION = "conversation"          # Current session
    AGENT_MEMORY = "agent_memory"          # Agent Memory Server
    STUDENT_PROFILE = "student_profile"    # Academic records
    COURSE_DATABASE = "course_database"    # Course information
    USER_PREFERENCES = "user_preferences"  # Explicit preferences
    BEHAVIORAL_DATA = "behavioral_data"    # Inferred from actions
    EXTERNAL_API = "external_api"          # External data sources

class ConflictResolutionStrategy(Enum):
    """Strategies for resolving conflicting information."""
    MOST_RECENT = "most_recent"            # Use newest information
    HIGHEST_PRIORITY = "highest_priority"  # Use most trusted source
    MOST_FREQUENT = "most_frequent"        # Use most commonly stated
    USER_EXPLICIT = "user_explicit"        # Prefer explicit user statements
    WEIGHTED_AVERAGE = "weighted_average"  # Combine based on weights
    CONTEXT_DEPENDENT = "context_dependent" # Depends on current situation

@dataclass
class ContextSource:
    """Represents a source of context information."""
    source_type: ContextSourceType
    source_id: str
    priority: float  # 0.0 to 1.0, higher = more trusted
    reliability: float  # 0.0 to 1.0, based on historical accuracy
    freshness_weight: float = 1.0  # How much recency matters
    
    def calculate_source_weight(self, age_hours: float = 0) -> float:
        """Calculate overall weight for this source."""
        # Base weight from priority and reliability
        base_weight = (self.priority + self.reliability) / 2
        
        # Apply freshness decay if age is provided
        if age_hours > 0 and self.freshness_weight > 0:
            # Exponential decay: weight decreases over time
            freshness_factor = math.exp(-age_hours / (24 * self.freshness_weight))
            return base_weight * freshness_factor
        
        return base_weight

@dataclass
class ContextItem:
    """Individual piece of context information."""
    id: str
    content: str
    source: ContextSource
    timestamp: datetime
    confidence: float = 1.0  # How confident we are in this information
    tags: List[str] = field(default_factory=list)
    metadata: Dict[str, Any] = field(default_factory=dict)
    
    def age_in_hours(self) -> float:
        """Calculate age of this context item in hours."""
        return (datetime.now() - self.timestamp).total_seconds() / 3600
    
    def get_effective_weight(self) -> float:
        """Get the effective weight considering source and age."""
        source_weight = self.source.calculate_source_weight(self.age_in_hours())
        return source_weight * self.confidence

# Define source configurations for the Redis University system
CONTEXT_SOURCES = {
    ContextSourceType.CONVERSATION: ContextSource(
        source_type=ContextSourceType.CONVERSATION,
        source_id="current_session",
        priority=0.9,  # High priority for current conversation
        reliability=0.8,  # Generally reliable but can have misunderstandings
        freshness_weight=2.0  # Very sensitive to recency
    ),
    
    ContextSourceType.AGENT_MEMORY: ContextSource(
        source_type=ContextSourceType.AGENT_MEMORY,
        source_id="agent_memory_server",
        priority=0.8,  # High priority for stored memories
        reliability=0.9,  # Very reliable, curated information
        freshness_weight=0.5  # Less sensitive to age
    ),
    
    ContextSourceType.STUDENT_PROFILE: ContextSource(
        source_type=ContextSourceType.STUDENT_PROFILE,
        source_id="academic_records",
        priority=1.0,  # Highest priority for official records
        reliability=0.95,  # Very reliable, official data
        freshness_weight=0.1  # Academic records don't change often
    ),
    
    ContextSourceType.USER_PREFERENCES: ContextSource(
        source_type=ContextSourceType.USER_PREFERENCES,
        source_id="explicit_preferences",
        priority=0.85,  # High priority for explicit user statements
        reliability=0.7,  # Users can change their minds
        freshness_weight=1.5  # Preferences can change over time
    ),
    
    ContextSourceType.BEHAVIORAL_DATA: ContextSource(
        source_type=ContextSourceType.BEHAVIORAL_DATA,
        source_id="inferred_behavior",
        priority=0.6,  # Lower priority for inferred data
        reliability=0.6,  # Less reliable, based on inference
        freshness_weight=1.0  # Moderately sensitive to recency
    )
}

print(f"✅ Context source framework initialized with {len(CONTEXT_SOURCES)} source types")

## Context Fusion Engine

Now let's create the main fusion engine that intelligently combines context:

In [ ]:
import math
from collections import defaultdict

class ContextFusionEngine:
    """Intelligent engine for fusing context from multiple sources."""
    
    def __init__(self):
        self.sources = CONTEXT_SOURCES
        self.fusion_stats = {
            "total_fusions": 0,
            "conflicts_resolved": 0,
            "sources_used": defaultdict(int)
        }
    
    async def fuse_context(self, 
                         context_items: List[ContextItem],
                         query_context: str = "",
                         max_items: int = 10) -> Dict[str, Any]:
        """Fuse context items from multiple sources into coherent context."""
        
        if not context_items:
            return {
                "fused_context": [],
                "conflicts_detected": [],
                "fusion_summary": "No context items to fuse"
            }
        
        # Step 1: Group items by topic/similarity
        topic_groups = self._group_by_topic(context_items)
        
        # Step 2: Detect and resolve conflicts within each group
        resolved_groups = []
        conflicts_detected = []
        
        for topic, items in topic_groups.items():
            if len(items) > 1:
                # Potential conflict - multiple items about same topic
                conflict_analysis = self._analyze_conflict(items, topic)
                if conflict_analysis["has_conflict"]:
                    conflicts_detected.append(conflict_analysis)
                
                # Resolve conflict
                resolved_item = self._resolve_conflict(items, query_context)
                resolved_groups.append(resolved_item)
            else:
                # No conflict, use the single item
                resolved_groups.extend(items)
        
        # Step 3: Rank and select final context items
        ranked_items = self._rank_context_items(resolved_groups, query_context)
        final_context = ranked_items[:max_items]
        
        # Step 4: Create fusion summary
        fusion_summary = self._create_fusion_summary(final_context, conflicts_detected)
        
        # Update statistics
        self._update_fusion_stats(final_context, conflicts_detected)
        
        return {
            "fused_context": final_context,
            "conflicts_detected": conflicts_detected,
            "fusion_summary": fusion_summary,
            "source_distribution": self._get_source_distribution(final_context)
        }
    
    def _group_by_topic(self, context_items: List[ContextItem]) -> Dict[str, List[ContextItem]]:
        """Group context items by topic/similarity."""
        # Simple topic grouping based on keywords
        # In production, you'd use semantic similarity
        
        topic_keywords = {
            "course_preferences": ["prefer", "like", "format", "online", "in-person", "hybrid"],
            "schedule_preferences": ["schedule", "time", "morning", "evening", "weekend"],
            "academic_progress": ["completed", "grade", "gpa", "credit", "semester"],
            "career_goals": ["career", "job", "work", "industry", "goal"],
            "course_interests": ["interested", "want to take", "considering", "planning"]
        }
        
        groups = defaultdict(list)
        
        for item in context_items:
            content_lower = item.content.lower()
            
            # Find best matching topic
            best_topic = "general"
            max_matches = 0
            
            for topic, keywords in topic_keywords.items():
                matches = sum(1 for keyword in keywords if keyword in content_lower)
                if matches > max_matches:
                    max_matches = matches
                    best_topic = topic
            
            groups[best_topic].append(item)
        
        return dict(groups)
    
    def _analyze_conflict(self, items: List[ContextItem], topic: str) -> Dict[str, Any]:
        """Analyze if items represent conflicting information."""
        
        # Simple conflict detection based on contradictory keywords
        conflict_patterns = {
            "course_preferences": [
                (["online", "remote"], ["in-person", "on-campus"]),
                (["easy", "simple"], ["challenging", "difficult"]),
                (["morning"], ["evening", "night"])
            ],
            "schedule_preferences": [
                (["morning", "early"], ["evening", "late"]),
                (["weekday"], ["weekend"]),
                (["flexible"], ["fixed", "strict"])
            ]
        }
        
        patterns = conflict_patterns.get(topic, [])
        conflicts_found = []
        
        for positive_keywords, negative_keywords in patterns:
            positive_items = []
            negative_items = []
            
            for item in items:
                content_lower = item.content.lower()
                
                if any(keyword in content_lower for keyword in positive_keywords):
                    positive_items.append(item)
                elif any(keyword in content_lower for keyword in negative_keywords):
                    negative_items.append(item)
            
            if positive_items and negative_items:
                conflicts_found.append({
                    "pattern": f"{positive_keywords} vs {negative_keywords}",
                    "positive_items": positive_items,
                    "negative_items": negative_items
                })
        
        return {
            "has_conflict": len(conflicts_found) > 0,
            "topic": topic,
            "conflicts": conflicts_found,
            "total_items": len(items)
        }
    
    def _resolve_conflict(self, 
                        items: List[ContextItem], 
                        query_context: str = "",
                        strategy: ConflictResolutionStrategy = ConflictResolutionStrategy.MOST_RECENT) -> ContextItem:
        """Resolve conflict between multiple context items."""
        
        if len(items) == 1:
            return items[0]
        
        if strategy == ConflictResolutionStrategy.MOST_RECENT:
            # Use the most recent item
            return max(items, key=lambda x: x.timestamp)
        
        elif strategy == ConflictResolutionStrategy.HIGHEST_PRIORITY:
            # Use item from highest priority source
            return max(items, key=lambda x: x.source.priority)
        
        elif strategy == ConflictResolutionStrategy.USER_EXPLICIT:
            # Prefer explicit user statements
            conversation_items = [item for item in items 
                                if item.source.source_type == ContextSourceType.CONVERSATION]
            if conversation_items:
                return max(conversation_items, key=lambda x: x.timestamp)
            else:
                return max(items, key=lambda x: x.get_effective_weight())
        
        else:
            # Default: use effective weight (combines source priority, reliability, and age)
            return max(items, key=lambda x: x.get_effective_weight())
    
    def _rank_context_items(self, items: List[ContextItem], query_context: str) -> List[ContextItem]:
        """Rank context items by relevance and importance."""
        
        def calculate_relevance_score(item: ContextItem) -> float:
            # Base score from effective weight
            base_score = item.get_effective_weight()
            
            # Boost score if relevant to current query
            if query_context:
                query_words = set(query_context.lower().split())
                item_words = set(item.content.lower().split())
                
                # Simple relevance boost based on word overlap
                overlap = len(query_words & item_words)
                if overlap > 0:
                    relevance_boost = min(overlap / len(query_words), 0.5)
                    base_score += relevance_boost
            
            return base_score
        
        # Sort by relevance score (descending)
        return sorted(items, key=calculate_relevance_score, reverse=True)
    
    def _create_fusion_summary(self, 
                             final_context: List[ContextItem], 
                             conflicts: List[Dict[str, Any]]) -> str:
        """Create a summary of the fusion process."""
        
        summary_parts = []
        
        # Context composition
        source_counts = defaultdict(int)
        for item in final_context:
            source_counts[item.source.source_type.value] += 1
        
        summary_parts.append(f"Fused {len(final_context)} context items from {len(source_counts)} sources")
        
        # Source breakdown
        if source_counts:
            source_breakdown = ", ".join([f"{count} from {source}" for source, count in source_counts.items()])
            summary_parts.append(f"Sources: {source_breakdown}")
        
        # Conflicts resolved
        if conflicts:
            summary_parts.append(f"Resolved {len(conflicts)} conflicts")
        
        return ". ".join(summary_parts)
    
    def _get_source_distribution(self, items: List[ContextItem]) -> Dict[str, int]:
        """Get distribution of sources in final context."""
        distribution = defaultdict(int)
        for item in items:
            distribution[item.source.source_type.value] += 1
        return dict(distribution)
    
    def _update_fusion_stats(self, final_context: List[ContextItem], conflicts: List[Dict[str, Any]]):
        """Update fusion statistics."""
        self.fusion_stats["total_fusions"] += 1
        self.fusion_stats["conflicts_resolved"] += len(conflicts)
        
        for item in final_context:
            self.fusion_stats["sources_used"][item.source.source_type.value] += 1
    
    def get_fusion_statistics(self) -> Dict[str, Any]:
        """Get fusion engine statistics."""
        return dict(self.fusion_stats)

# Initialize the fusion engine
fusion_engine = ContextFusionEngine()

print("✅ Context fusion engine initialized")

## Demonstration: Context Fusion in Action

Let's create sample context items from different sources and see how fusion handles conflicts:

In [ ]:
# Create sample context items with conflicts
def create_sample_context_items() -> List[ContextItem]:
    """Create sample context items from different sources with some conflicts."""
    
    base_time = datetime.now()
    items = []
    
    # Recent conversation - student says they prefer online
    items.append(ContextItem(
        id="conv_001",
        content="I prefer online courses because of my work schedule",
        source=CONTEXT_SOURCES[ContextSourceType.CONVERSATION],
        timestamp=base_time - timedelta(minutes=5),
        confidence=0.9,
        tags=["preference", "format"]
    ))
    
    # Agent memory - older preference for in-person
    items.append(ContextItem(
        id="memory_001",
        content="Student previously expressed preference for in-person classes for better interaction",
        source=CONTEXT_SOURCES[ContextSourceType.AGENT_MEMORY],
        timestamp=base_time - timedelta(days=30),
        confidence=0.8,
        tags=["preference", "format", "historical"]
    ))
    
    # Student profile - academic standing
    items.append(ContextItem(
        id="profile_001",
        content="Student has completed CS201 with grade A and CS301 with grade B+",
        source=CONTEXT_SOURCES[ContextSourceType.STUDENT_PROFILE],
        timestamp=base_time - timedelta(days=60),
        confidence=1.0,
        tags=["academic", "progress", "grades"]
    ))
    
    # Behavioral data - inferred from actions
    items.append(ContextItem(
        id="behavior_001",
        content="Student consistently searches for evening and weekend course sections",
        source=CONTEXT_SOURCES[ContextSourceType.BEHAVIORAL_DATA],
        timestamp=base_time - timedelta(days=7),
        confidence=0.7,
        tags=["schedule", "preference", "inferred"]
    ))
    
    # User preferences - explicit setting
    items.append(ContextItem(
        id="pref_001",
        content="User profile setting: Preferred difficulty level = Intermediate",
        source=CONTEXT_SOURCES[ContextSourceType.USER_PREFERENCES],
        timestamp=base_time - timedelta(days=14),
        confidence=0.9,
        tags=["difficulty", "preference", "explicit"]
    ))
    
    # Recent conversation - conflicting schedule preference
    items.append(ContextItem(
        id="conv_002",
        content="I actually prefer morning classes now, I'm more focused then",
        source=CONTEXT_SOURCES[ContextSourceType.CONVERSATION],
        timestamp=base_time - timedelta(minutes=10),
        confidence=0.8,
        tags=["schedule", "preference", "morning"]
    ))
    
    # Agent memory - career interest
    items.append(ContextItem(
        id="memory_002",
        content="Student expressed strong interest in machine learning and AI careers",
        source=CONTEXT_SOURCES[ContextSourceType.AGENT_MEMORY],
        timestamp=base_time - timedelta(days=20),
        confidence=0.9,
        tags=["career", "interest", "ai", "ml"]
    ))
    
    # Behavioral data - course viewing patterns
    items.append(ContextItem(
        id="behavior_002",
        content="Student has viewed CS401 (Machine Learning) details 5 times in past week",
        source=CONTEXT_SOURCES[ContextSourceType.BEHAVIORAL_DATA],
        timestamp=base_time - timedelta(days=2),
        confidence=0.8,
        tags=["course", "interest", "ml", "behavior"]
    ))
    
    return items

# Create sample data
sample_context_items = create_sample_context_items()

print(f"📚 Created {len(sample_context_items)} sample context items")
print("\n📋 Context Items Overview:")
for item in sample_context_items:
    age_hours = item.age_in_hours()
    weight = item.get_effective_weight()
    print(f"   • [{item.source.source_type.value}] {item.content[:50]}... (Age: {age_hours:.1f}h, Weight: {weight:.3f})")

## Testing Context Fusion

Let's test the fusion engine with different scenarios:

In [ ]:
# Test context fusion with different query contexts
print("🧪 Testing Context Fusion")
print("=" * 60)

test_scenarios = [
    {
        "name": "Course Format Inquiry",
        "query": "What course format does the student prefer?",
        "max_items": 5
    },
    {
        "name": "Academic Planning",
        "query": "Help me plan courses for machine learning specialization",
        "max_items": 6
    },
    {
        "name": "Schedule Planning",
        "query": "What time of day does the student prefer for classes?",
        "max_items": 4
    }
]

fusion_results = []

for scenario in test_scenarios:
    print(f"\n🎯 Scenario: {scenario['name']}")
    print(f"📝 Query: '{scenario['query']}'")
    print("-" * 50)
    
    # Perform fusion
    result = await fusion_engine.fuse_context(
        context_items=sample_context_items,
        query_context=scenario['query'],
        max_items=scenario['max_items']
    )
    
    fusion_results.append(result)
    
    # Display results
    print(f"📊 Fusion Summary: {result['fusion_summary']}")
    
    if result['conflicts_detected']:
        print(f"⚠️  Conflicts Detected: {len(result['conflicts_detected'])}")
        for i, conflict in enumerate(result['conflicts_detected'], 1):
            print(f"   {i}. {conflict['topic']}: {conflict['pattern']}")
    
    print(f"\n🎯 Final Fused Context ({len(result['fused_context'])} items):")
    for i, item in enumerate(result['fused_context'], 1):
        source_type = item.source.source_type.value
        weight = item.get_effective_weight()
        print(f"   {i}. [{source_type}] {item.content[:60]}... (Weight: {weight:.3f})")
    
    print(f"\n📈 Source Distribution: {result['source_distribution']}")
    print("=" * 50)

## Conflict Resolution Strategies

Let's test different conflict resolution strategies:

In [ ]:
# Test different conflict resolution strategies
print("🔄 Testing Conflict Resolution Strategies")
print("=" * 60)

# Create a specific conflict scenario
conflicting_items = [
    ContextItem(
        id="recent_pref",
        content="I prefer online courses now due to my work schedule",
        source=CONTEXT_SOURCES[ContextSourceType.CONVERSATION],
        timestamp=datetime.now() - timedelta(minutes=5),
        confidence=0.9
    ),
    ContextItem(
        id="old_pref",
        content="Student prefers in-person classes for better interaction",
        source=CONTEXT_SOURCES[ContextSourceType.AGENT_MEMORY],
        timestamp=datetime.now() - timedelta(days=30),
        confidence=0.8
    ),
    ContextItem(
        id="profile_pref",
        content="Profile setting: Course format preference = Hybrid",
        source=CONTEXT_SOURCES[ContextSourceType.USER_PREFERENCES],
        timestamp=datetime.now() - timedelta(days=14),
        confidence=0.9
    )
]

strategies_to_test = [
    ConflictResolutionStrategy.MOST_RECENT,
    ConflictResolutionStrategy.HIGHEST_PRIORITY,
    ConflictResolutionStrategy.USER_EXPLICIT
]

print("📝 Conflicting Items:")
for i, item in enumerate(conflicting_items, 1):
    age_hours = item.age_in_hours()
    weight = item.get_effective_weight()
    print(f"   {i}. [{item.source.source_type.value}] {item.content} (Age: {age_hours:.1f}h, Weight: {weight:.3f})")

print("\n🔧 Testing Resolution Strategies:")
for strategy in strategies_to_test:
    print(f"\n🎯 Strategy: {strategy.value}")
    
    resolved_item = fusion_engine._resolve_conflict(
        conflicting_items, 
        "What course format should I recommend?",
        strategy
    )
    
    print(f"   Winner: [{resolved_item.source.source_type.value}] {resolved_item.content}")
    print(f"   Reason: {strategy.value} strategy selected this item")

print("\n💡 Strategy Comparison:")
print("   • MOST_RECENT: Prioritizes newest information")
print("   • HIGHEST_PRIORITY: Uses source priority (Student Profile > Conversation > Memory)")
print("   • USER_EXPLICIT: Prefers direct user statements from conversation")

## Integration with Redis University Agent

Let's see how to integrate context fusion with your existing agent:

In [ ]:
# Enhanced agent with context fusion
class FusionEnhancedUniversityAgent:
    """Redis University Agent enhanced with context fusion capabilities."""
    
    def __init__(self, student_id: str):
        self.student_id = student_id
        self.fusion_engine = ContextFusionEngine()
        self.course_manager = CourseManager()
        
        # Simulated data sources (in real implementation, these would be actual connections)
        self.data_sources = {
            "agent_memory": self._get_agent_memory_context,
            "student_profile": self._get_student_profile_context,
            "conversation": self._get_conversation_context,
            "behavioral": self._get_behavioral_context
        }
    
    async def process_query_with_fusion(self, query: str, conversation_history: List[str] = None) -> Dict[str, Any]:
        """Process query using context fusion from multiple sources."""
        
        # Step 1: Gather context from all sources
        all_context_items = []
        
        for source_name, source_func in self.data_sources.items():
            try:
                source_items = await source_func(query, conversation_history)
                all_context_items.extend(source_items)
                print(f"✅ Gathered {len(source_items)} items from {source_name}")
            except Exception as e:
                print(f"⚠️  Failed to gather from {source_name}: {e}")
        
        # Step 2: Fuse context intelligently
        fusion_result = await self.fusion_engine.fuse_context(
            context_items=all_context_items,
            query_context=query,
            max_items=8
        )
        
        # Step 3: Generate response using fused context
        response = await self._generate_response_with_context(
            query, fusion_result['fused_context']
        )
        
        return {
            "query": query,
            "response": response,
            "fusion_summary": fusion_result['fusion_summary'],
            "conflicts_resolved": len(fusion_result['conflicts_detected']),
            "context_sources": fusion_result['source_distribution'],
            "total_context_items": len(fusion_result['fused_context'])
        }
    
    async def _get_agent_memory_context(self, query: str, conversation_history: List[str] = None) -> List[ContextItem]:
        """Get context from Agent Memory Server."""
        # Simulate Agent Memory Server retrieval
        memory_items = [
            ContextItem(
                id="memory_academic",
                content="Student has strong background in programming and mathematics",
                source=CONTEXT_SOURCES[ContextSourceType.AGENT_MEMORY],
                timestamp=datetime.now() - timedelta(days=10),
                confidence=0.9
            ),
            ContextItem(
                id="memory_career",
                content="Student expressed interest in AI and machine learning career paths",
                source=CONTEXT_SOURCES[ContextSourceType.AGENT_MEMORY],
                timestamp=datetime.now() - timedelta(days=15),
                confidence=0.8
            )
        ]
        return memory_items
    
    async def _get_student_profile_context(self, query: str, conversation_history: List[str] = None) -> List[ContextItem]:
        """Get context from student academic profile."""
        # Simulate student profile data
        profile_items = [
            ContextItem(
                id="profile_academic",
                content="Current GPA: 3.7, Major: Computer Science, Credits: 45/120",
                source=CONTEXT_SOURCES[ContextSourceType.STUDENT_PROFILE],
                timestamp=datetime.now() - timedelta(days=1),
                confidence=1.0
            )
        ]
        return profile_items
    
    async def _get_conversation_context(self, query: str, conversation_history: List[str] = None) -> List[ContextItem]:
        """Get context from current conversation."""
        conversation_items = []
        
        if conversation_history:
            for i, message in enumerate(conversation_history[-3:]):  # Last 3 messages
                conversation_items.append(ContextItem(
                    id=f"conv_{i}",
                    content=message,
                    source=CONTEXT_SOURCES[ContextSourceType.CONVERSATION],
                    timestamp=datetime.now() - timedelta(minutes=5*(len(conversation_history)-i)),
                    confidence=0.9
                ))
        
        # Add current query
        conversation_items.append(ContextItem(
            id="current_query",
            content=query,
            source=CONTEXT_SOURCES[ContextSourceType.CONVERSATION],
            timestamp=datetime.now(),
            confidence=1.0
        ))
        
        return conversation_items
    
    async def _get_behavioral_context(self, query: str, conversation_history: List[str] = None) -> List[ContextItem]:
        """Get context from behavioral data."""
        # Simulate behavioral insights
        behavioral_items = [
            ContextItem(
                id="behavior_search",
                content="Student frequently searches for machine learning and AI courses",
                source=CONTEXT_SOURCES[ContextSourceType.BEHAVIORAL_DATA],
                timestamp=datetime.now() - timedelta(days=3),
                confidence=0.7
            )
        ]
        return behavioral_items
    
    async def _generate_response_with_context(self, query: str, context_items: List[ContextItem]) -> str:
        """Generate response using fused context."""
        # Simulate response generation (in real implementation, use LLM)
        context_summary = "\n".join([f"- {item.content}" for item in context_items[:3]])
        
        return f"Based on your profile and preferences, here's my recommendation for '{query}'. Key context considered:\n{context_summary}"

# Test the enhanced agent
enhanced_agent = FusionEnhancedUniversityAgent("test_student")

print("🤖 Testing Fusion-Enhanced University Agent")
print("=" * 60)

test_query = "What machine learning courses should I take next semester?"
conversation_history = [
    "I'm interested in AI and data science careers",
    "I prefer online courses due to work schedule",
    "I've completed CS201 and MATH201"
]

print(f"📝 Query: {test_query}")
print(f"📚 Conversation History: {len(conversation_history)} previous messages")

result = await enhanced_agent.process_query_with_fusion(test_query, conversation_history)

print(f"\n📊 Fusion Results:")
print(f"   Fusion Summary: {result['fusion_summary']}")
print(f"   Conflicts Resolved: {result['conflicts_resolved']}")
print(f"   Context Sources: {result['context_sources']}")
print(f"   Total Context Items: {result['total_context_items']}")

print(f"\n🤖 Agent Response:")
print(result['response'])

## 🧪 Hands-on Exercise: Design Your Fusion Strategy

Now it's your turn to experiment with context fusion strategies:

In [ ]:
# Exercise: Create your own context fusion strategy
print("🧪 Exercise: Design Your Context Fusion Strategy")
print("=" * 60)

# TODO: Create a domain-specific fusion strategy for academic advising
class AcademicAdvisingFusionStrategy:
    """Specialized fusion strategy for academic advising scenarios."""
    
    def __init__(self):
        self.academic_priorities = {
            "graduation_requirements": 1.0,  # Highest priority
            "prerequisite_completion": 0.95,
            "gpa_maintenance": 0.9,
            "career_alignment": 0.8,
            "schedule_preferences": 0.6,
            "format_preferences": 0.5  # Lowest priority
        }
    
    def categorize_context_item(self, item: ContextItem) -> str:
        """Categorize context item by academic importance."""
        content_lower = item.content.lower()
        
        if any(word in content_lower for word in ["graduation", "degree", "requirement", "credit"]):
            return "graduation_requirements"
        elif any(word in content_lower for word in ["prerequisite", "completed", "grade", "gpa"]):
            return "prerequisite_completion"
        elif any(word in content_lower for word in ["career", "job", "industry", "goal"]):
            return "career_alignment"
        elif any(word in content_lower for word in ["schedule", "time", "morning", "evening"]):
            return "schedule_preferences"
        elif any(word in content_lower for word in ["online", "in-person", "hybrid", "format"]):
            return "format_preferences"
        else:
            return "gpa_maintenance"  # Default category
    
    def calculate_academic_weight(self, item: ContextItem) -> float:
        """Calculate weight based on academic importance."""
        category = self.categorize_context_item(item)
        academic_priority = self.academic_priorities.get(category, 0.5)
        
        # Combine with original effective weight
        base_weight = item.get_effective_weight()
        
        # Academic priority acts as a multiplier
        return base_weight * academic_priority
    
    def resolve_academic_conflict(self, items: List[ContextItem]) -> ContextItem:
        """Resolve conflicts using academic priorities."""
        if len(items) == 1:
            return items[0]
        
        # Calculate academic weights for all items
        weighted_items = [(item, self.calculate_academic_weight(item)) for item in items]
        
        # Sort by academic weight (descending)
        weighted_items.sort(key=lambda x: x[1], reverse=True)
        
        return weighted_items[0][0]  # Return item with highest academic weight
    
    def create_academic_fusion_summary(self, items: List[ContextItem]) -> str:
        """Create summary focused on academic decision factors."""
        categories = defaultdict(list)
        
        for item in items:
            category = self.categorize_context_item(item)
            categories[category].append(item)
        
        summary_parts = []
        
        # Prioritize summary by academic importance
        for category in sorted(categories.keys(), key=lambda x: self.academic_priorities.get(x, 0), reverse=True):
            item_count = len(categories[category])
            if item_count > 0:
                summary_parts.append(f"{item_count} {category.replace('_', ' ')} factors")
        
        return f"Academic fusion: {', '.join(summary_parts)}"

# Test the academic fusion strategy
academic_fusion = AcademicAdvisingFusionStrategy()

# Create academic-focused context items
academic_context_items = [
    ContextItem(
        id="req_001",
        content="Student needs 6 more core CS courses to meet graduation requirements",
        source=CONTEXT_SOURCES[ContextSourceType.STUDENT_PROFILE],
        timestamp=datetime.now() - timedelta(days=1),
        confidence=1.0
    ),
    ContextItem(
        id="pref_001",
        content="I prefer online courses for convenience",
        source=CONTEXT_SOURCES[ContextSourceType.CONVERSATION],
        timestamp=datetime.now() - timedelta(minutes=5),
        confidence=0.8
    ),
    ContextItem(
        id="career_001",
        content="Student wants to pursue machine learning career requiring advanced math",
        source=CONTEXT_SOURCES[ContextSourceType.AGENT_MEMORY],
        timestamp=datetime.now() - timedelta(days=10),
        confidence=0.9
    ),
    ContextItem(
        id="prereq_001",
        content="Student has completed CS201 and MATH201, eligible for CS301",
        source=CONTEXT_SOURCES[ContextSourceType.STUDENT_PROFILE],
        timestamp=datetime.now() - timedelta(days=30),
        confidence=1.0
    )
]

print("\n🎯 Testing Academic Fusion Strategy:")
print("\n📚 Academic Context Items:")
for item in academic_context_items:
    category = academic_fusion.categorize_context_item(item)
    academic_weight = academic_fusion.calculate_academic_weight(item)
    print(f"   • [{category}] {item.content[:50]}... (Weight: {academic_weight:.3f})")

# Test conflict resolution
print("\n🔄 Testing Academic Conflict Resolution:")
conflicting_academic_items = [
    academic_context_items[1],  # Format preference (low priority)
    academic_context_items[0],  # Graduation requirement (high priority)
]

resolved_item = academic_fusion.resolve_academic_conflict(conflicting_academic_items)
print(f"   Winner: {resolved_item.content[:60]}...")
print(f"   Reason: Academic priority system favored graduation requirements over preferences")

# Create fusion summary
fusion_summary = academic_fusion.create_academic_fusion_summary(academic_context_items)
print(f"\n📊 Academic Fusion Summary: {fusion_summary}")

print("\n🤔 Reflection Questions:")
print("1. How does academic prioritization change fusion decisions?")
print("2. When should student preferences override academic requirements?")
print("3. How would you handle conflicts between career goals and graduation timeline?")
print("4. What other domain-specific fusion strategies would be useful?")

print("\n🔧 Your Turn: Try These Modifications:")
print("   • Create fusion strategies for different student types (part-time, graduate, etc.)")
print("   • Add temporal reasoning (semester planning vs. long-term goals)")
print("   • Implement confidence-based fusion weighting")
print("   • Add user feedback to improve fusion decisions")
print("   • Create fusion strategies for different query types")

## Key Takeaways

From this exploration of context fusion, you've learned:

### 🎯 **Core Concepts**
- **Multi-source context** requires intelligent fusion to avoid conflicts
- **Source prioritization** based on reliability, recency, and domain importance
- **Conflict resolution** strategies for handling contradictory information
- **Temporal awareness** for managing information freshness and decay

### 🛠️ **Implementation Patterns**
- **Source weighting** combining priority, reliability, and freshness
- **Conflict detection** using pattern matching and semantic analysis
- **Resolution strategies** from simple (most recent) to complex (weighted fusion)
- **Domain-specific fusion** for academic advising scenarios

### 📊 **Fusion Benefits**
- **Coherent context** from fragmented information sources
- **Conflict resolution** prevents contradictory recommendations
- **Source transparency** shows where information comes from
- **Adaptive weighting** based on query context and domain priorities

### 🔄 **Fusion Strategies**
- **Most Recent**: Prioritize newest information
- **Highest Priority**: Trust most reliable sources
- **User Explicit**: Prefer direct user statements
- **Academic Priority**: Domain-specific importance weighting
- **Context Dependent**: Adapt strategy based on query type

### 🎓 **Academic Applications**
- **Graduation requirements** take priority over preferences
- **Prerequisites** must be considered before recommendations
- **Career alignment** balances with academic constraints
- **Student preferences** matter but don't override requirements

### 🚀 **Next Steps**
In the final notebook of Section 5, we'll explore **Context Validation & Health Monitoring** - how to detect context quality issues, monitor performance, and maintain context health in production systems.

The fusion techniques you've learned provide the foundation for creating coherent, reliable context from multiple information sources.

---

**Ready to continue?** Move on to `06_context_validation.ipynb` to learn about context quality assurance and health monitoring!